In [ ]:
# Import xml parser
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import fuzzy_pandas  as fpd

# Enter the file name/well name
print('Enter file name: ')
file_name = input()
print('file name: ' + file_name)
print('Enter the most recent WBS')
wbs = input()
print('Recent WBS:' + wbs)

tree = ET.parse(r"C:\Users\\Documents"+str(file_name)+".edm.xml")

root = tree.getroot()
elemlist = []

#CD_WELL (get the well name and API no)
well_cols = ["WELL_ID","API_NO","WELL_LEGAL_NAME"]
well_rows=[]
for i in root.findall('CD_WELL'):
        WELL_ID = i.get('WELL_ID')
        API_NO = i.get('API_NO')
        SPUD_DATE = i.get('SPUD_DATE')
        WELL_LEGAL_NAME = i.get('WELL_LEGAL_NAME')   
        well_rows.append({"WELL_ID": WELL_ID,"API_NO": API_NO,"WELL_LEGAL_NAME": WELL_LEGAL_NAME})
wellname = pd.DataFrame(well_rows, columns = well_cols).drop_duplicates()
# print(wellname)
# DM_PIPE_DATA (Get the size of all pipes used in the well)
pipe_cols = ['WELL_ID','EVENT_ID','PIPE_RUN_ID','PIPE_DATA_ID','SECT_TYPE_CODE','DESCRIPTION','NOMINAL_SIZE','THREAD_TYPE']
pipe_rows = []
for i in root.findall('DM_PIPE_DATA'):
    WELL_ID = i.get('WELL_ID')
    EVENT_ID = i.get('EVENT_ID')
    PIPE_RUN_ID = i.get('PIPE_RUN_ID')
    PIPE_DATA_ID = i.get('PIPE_DATA_ID')
    SECT_TYPE_CODE = i.get('SECT_TYPE_CODE')
    DESCRIPTION = i.get('DESCRIPTION')
    NOMINAL_SIZE = i.get('NOMINAL_SIZE')
    THREAD_TYPE = i.get('THREAD_TYPE')
    pipe_rows.append({'WELL_ID':WELL_ID,'EVENT_ID':EVENT_ID,'PIPE_RUN_ID':PIPE_RUN_ID,'PIPE_DATA_ID':PIPE_DATA_ID,'SECT_TYPE_CODE':SECT_TYPE_CODE,'DESCRIPTION':DESCRIPTION,'NOMINAL_SIZE':NOMINAL_SIZE,'THREAD_TYPE':THREAD_TYPE})
pipedata = pd.DataFrame(pipe_rows, columns = pipe_cols).drop_duplicates()
# pipedata.head()
#DM_REPORT_JOURNAL (get the list of all relevant reports)
reports_cols = ['REPORT_JOURNAL_ID','EVENT_ID','DESCRIPTION','CREATE_DATE','ENTITY_TYPE','EVENT_CODE','REPORT_ALIAS']
reports_rows = []
events= ['DON','CON']
desc =['PLUG','LINER','CBP','HANGER','FISH','RUN']
alias = ['CASING','GEN_TRAN','GEN_PIPE','WB_EQUIP']
desc_pattern = '|'.join(desc)
for i in root.findall('DM_REPORT_JOURNAL'):
    REPORT_JOURNAL_ID = i.get('REPORT_JOURNAL_ID')
    EVENT_ID = i.get('EVENT_ID')
    DESCRIPTION = i.get('DESCRIPTION')
    CREATE_DATE = i.get('CREATE_DATE')
    ENTITY_TYPE = i.get('ENTITY_TYPE')
    EVENT_CODE = i.get('EVENT_CODE')
    REPORT_ALIAS = i.get('REPORT_ALIAS')
    reports_rows.append({'REPORT_JOURNAL_ID':REPORT_JOURNAL_ID,'EVENT_ID':EVENT_ID,'DESCRIPTION':DESCRIPTION,'CREATE_DATE':CREATE_DATE,'ENTITY_TYPE':ENTITY_TYPE,'EVENT_CODE':EVENT_CODE,'REPORT_ALIAS':REPORT_ALIAS})
reports = pd.DataFrame(reports_rows, columns = reports_cols).drop_duplicates()
reports = reports[reports.EVENT_CODE.isin(events)]
reports = reports[reports.REPORT_ALIAS.isin(alias)]
reports = reports.loc[~(reports['DESCRIPTION'].str.contains(desc_pattern, case=False))]
# reports.head()
well = wellname.WELL_LEGAL_NAME[0]
#DM_PIPE_RUN (shows tally type, we need to get the offload)
sumofflist_cols = ["PIPE_RUN_ID","TOTAL_LENGTH","RUN_TALLY_TYPE","RUN_LENGTH","WELLBORE_ID"]
sumofflist_rows=[]
ctlg = ['PUP','WELLHEAD','LANDING','MARKER','CROSS']
ctlg_desc = '|'.join(ctlg)
for i in root.findall('DM_PIPE_RUN'):
    PIPE_RUN_ID = i.get('PIPE_RUN_ID')
    TOTAL_LENGTH = i.get('TOTAL_LENGTH')
    RUN_TALLY_TYPE = i.get('RUN_TALLY_TYPE')
    RUN_LENGTH = i.get('RUN_LENGTH')
    WELLBORE_ID = i.get('WELLBORE_ID')

    sumofflist_rows.append({"PIPE_RUN_ID": PIPE_RUN_ID, "TOTAL_LENGTH": TOTAL_LENGTH,"RUN_TALLY_TYPE": RUN_TALLY_TYPE, "RUN_LENGTH": RUN_LENGTH, "WELLBORE_ID": WELLBORE_ID})
    
sumoff = pd.DataFrame(sumofflist_rows, columns = sumofflist_cols).drop_duplicates()
# sumoff.head()
#DM_PIPE_TALLY (get the sum of joints and length per pipe, need to filter RUN_TALLY_TYPE = OFF)
#Verified 
#Tubing ofthen doesn't have offload or transfer report 
tally_cols = ['WELL_ID','EVENT_ID','PIPE_RUN_ID','PIPE_TALLY_ID','PIPE_DATA_ID','LENGTH','JOINT_NUMBER']
tally_rows = []
for i in root.findall('DM_PIPE_TALLY'):
    WELL_ID = i.get('WELL_ID')
    EVENT_ID = i.get('EVENT_ID')
    PIPE_RUN_ID = i.get('PIPE_RUN_ID')
    PIPE_TALLY_ID = i.get('PIPE_TALLY_ID')
    PIPE_DATA_ID = i.get('PIPE_DATA_ID')
    LENGTH = i.get('LENGTH')
    JOINT_NUMBER = i.get('JOINT_NUMBER')
    
    tally_rows.append({'WELL_ID':WELL_ID,'EVENT_ID':EVENT_ID,'PIPE_RUN_ID':PIPE_RUN_ID,'PIPE_TALLY_ID':PIPE_TALLY_ID,'PIPE_DATA_ID':PIPE_DATA_ID,'LENGTH':LENGTH,'JOINT_NUMBER':JOINT_NUMBER})
tally = pd.DataFrame(tally_rows, columns = tally_cols).drop_duplicates()
tally['LENGTH'] = tally['LENGTH'].astype(float).round(2)
tally = pd.merge(tally,wellname[["WELL_ID","WELL_LEGAL_NAME"]], on ='WELL_ID')
tally = pd.merge(tally,pipedata[['PIPE_DATA_ID','SECT_TYPE_CODE','DESCRIPTION','NOMINAL_SIZE','THREAD_TYPE']], on ='PIPE_DATA_ID')
tally = pd.merge(tally, sumoff[['PIPE_RUN_ID','RUN_TALLY_TYPE']], on = 'PIPE_RUN_ID')
tally = tally[tally["RUN_TALLY_TYPE"] == 'OFF']
tally['THREAD_TYPE'].fillna("NA",inplace = True)
tally['NOMINAL_SIZE'].fillna("NA",inplace = True)
tally['NO_JOINTS'] = '1'
tally = tally.loc[~(tally['DESCRIPTION'].str.contains(ctlg_desc,case=False))]
tally['KEYWORD'] = tally['SECT_TYPE_CODE']+' '+tally['NOMINAL_SIZE'] + ' ' + tally['THREAD_TYPE']
offload = tally.groupby(['WELL_LEGAL_NAME','NOMINAL_SIZE','DESCRIPTION']).agg({'LENGTH':'sum', 'NO_JOINTS':np.count_nonzero })
# offload

#DM_TRANSFER_SUMMARY (Get the summary transfer data)
tfrsum_cols = ['WELL_ID','EVENT_ID','TRANSFER_ID','TRANSFER_SUMMARY_ID','ORIGIN','DESTINATION','REASON','AGENT']
tfrsum_rows = []
desc = ['PUP','WELLHEAD','LANDING','MARKER','CROSS']
desc_remove = '|'.join(desc)
origin = ['PREMIER','TUBOSCOPE','TENARIS']
for i in root.findall('DM_TRANSFER_SUMMARY'):
    WELL_ID = i.get('WELL_ID')
    EVENT_ID = i.get('EVENT_ID')
    TRANSFER_ID = i.get('TRANSFER_ID')
    TRANSFER_SUMMARY_ID = i.get('TRANSFER_SUMMARY_ID')
    ORIGIN = i.get('ORIGIN')
    DESTINATION = i.get('DESTINATION')
    REASON = i.get('REASON')
    AGENT = i.get('AGENT')
    
    
    tfrsum_rows.append({'WELL_ID': WELL_ID,'EVENT_ID': EVENT_ID,'TRANSFER_ID': TRANSFER_ID,
                        'TRANSFER_SUMMARY_ID': TRANSFER_SUMMARY_ID,'ORIGIN': ORIGIN,
                        'DESTINATION': DESTINATION,'REASON': REASON, 'AGENT': AGENT})
tfrsum = pd.DataFrame(tfrsum_rows, columns = tfrsum_cols).drop_duplicates()

# tfrsum
#DM_TRANSFER_DETAIL (Get all Transfer Data)
transfer_cols = ['WELL_ID','EVENT_ID','TRANSFER_ID','TRANSFER_SUMMARY_ID','TRANSFER_DETAIL_ID','DESCRIPTION','QUANTITY','UNITS','MANUFACTURER_SERIAL_NUMBER']
transfer_rows = []
for i in root.findall('DM_TRANSFER_DETAIL'):
    WELL_ID = i.get('WELL_ID')
    EVENT_ID = i.get('EVENT_ID')
    TRANSFER_ID = i.get('TRANSFER_ID')
    TRANSFER_SUMMARY_ID = i.get('TRANSFER_SUMMARY_ID')
    TRANSFER_DETAIL_ID = i.get('TRANSFER_DETAIL_ID')
    DESCRIPTION = i.get('DESCRIPTION')
    QUANTITY = i.get('QUANTITY')
    UNITS = i.get('UNITS')
    MANUFACTURER_SERIAL_NUMBER = i.get('MANUFACTURER_SERIAL_NUMBER')
    
    transfer_rows.append({'WELL_ID': WELL_ID,'EVENT_ID':EVENT_ID,'TRANSFER_ID':TRANSFER_ID,'TRANSFER_SUMMARY_ID':TRANSFER_SUMMARY_ID,'TRANSFER_DETAIL_ID':TRANSFER_DETAIL_ID,'DESCRIPTION':DESCRIPTION,'QUANTITY':QUANTITY,'UNITS':UNITS,'MANUFACTURER_SERIAL_NUMBER': MANUFACTURER_SERIAL_NUMBER})
transfer = pd.DataFrame(transfer_rows, columns = transfer_cols).drop_duplicates()
transfer['QUANTITY'] = transfer['QUANTITY'].astype(float)
transfer = pd.merge(transfer,wellname[["WELL_ID","WELL_LEGAL_NAME"]], on ='WELL_ID')
transfer = pd.merge(transfer,tfrsum[['EVENT_ID','TRANSFER_ID','TRANSFER_SUMMARY_ID','ORIGIN','DESTINATION','REASON','AGENT']],on = ['EVENT_ID','TRANSFER_ID','TRANSFER_SUMMARY_ID'] )
transfer['in/out'] = np.where(transfer['DESTINATION']==wellname.WELL_LEGAL_NAME[0], 'transfer in', 'transfer out')
transfer['DESCRIPTION'] = transfer['DESCRIPTION'].str.replace(',','').str.replace('40#','').str.replace('BUTTRESS','').str.replace('23#','').str.replace('5-1/2"','5.5').str.replace('9-5/8"','9.625')
transfer = transfer.loc[~(transfer['DESCRIPTION'].str.contains(desc_remove,case=False))]
transfer = transfer[~(transfer.ORIGIN.isin(origin))]
transferdata = transfer.groupby(['WELL_LEGAL_NAME','in/out','ORIGIN','DESTINATION','DESCRIPTION','REASON', 'MANUFACTURER_SERIAL_NUMBER']).agg({'QUANTITY': 'sum'})     
transferdata = transferdata.reset_index()
# transferdata

# CD_ASSEMBLY
asm_cols = ['ASSEMBLY_ID','REPORT_JOURNAL_ID','STRING_TYPE','DATE_REPORT','ASSEMBLY_NAME','ASSEMBLY_SIZE','LENGTH_TOTAL','EVENT_ID']
asm_rows = []
for i in root.findall('CD_ASSEMBLY'):
    ASSEMBLY_ID = i.get('ASSEMBLY_ID')
    REPORT_JOURNAL_ID = i.get('REPORT_JOURNAL_ID')
    STRING_TYPE = i.get('STRING_TYPE')
    DATE_REPORT = i.get('DATE_REPORT')
    ASSEMBLY_NAME = i.get('ASSEMBLY_NAME')
    ASSEMBLY_SIZE = i.get('ASSEMBLY_SIZE')
    LENGTH_TOTAL = i.get('LENGTH_TOTAL')
    EVENT_ID = i.get('EVENT_ID')
    
    asm_rows.append({'ASSEMBLY_ID':ASSEMBLY_ID,'REPORT_JOURNAL_ID':REPORT_JOURNAL_ID,'STRING_TYPE':STRING_TYPE,'DATE_REPORT':DATE_REPORT,'ASSEMBLY_NAME':ASSEMBLY_NAME,'ASSEMBLY_SIZE':ASSEMBLY_SIZE,'LENGTH_TOTAL':LENGTH_TOTAL,'EVENT_ID':EVENT_ID})
sect1 = ['CAS','TBG']
asm_name = ['CONDUCTOR','PACKER']
ctlg = ['PUP','WELLHEAD','LANDING','MARKER','CROSS']
asm_name_desc = '|'.join(asm_name)
ctlg_desc = '|'.join(ctlg)
asm = pd.DataFrame(asm_rows, columns = asm_cols).drop_duplicates()
asm = pd.merge(asm,reports[['REPORT_JOURNAL_ID','EVENT_ID','DESCRIPTION','ENTITY_TYPE']], on = ['REPORT_JOURNAL_ID','EVENT_ID'])
# asm
#CD_ASSEMBLY_COMP(Burried report for Casing and Tubing) 
asmbly_cols = ['WELL_ID','ASSEMBLY_ID','ASSEMBLY_COMP_ID','SECT_TYPE_CODE','CATALOG_KEY_DESC','COMP_NAME',
              'DESCRIPTION','LENGTH','OD_BODY','SEQUENCE_NO','COMP_TYPE_CODE','CREATE_USER_ID',
               'CREATE_DATE','GRADE','JOINTS','CONNECTION_NAME','NOMINAL_SIZE']
asmbly_rows = []
for i in root.findall('CD_ASSEMBLY_COMP'):
    WELL_ID = i.get('WELL_ID')
    ASSEMBLY_ID = i.get('ASSEMBLY_ID')
    ASSEMBLY_COMP_ID = i.get('ASSEMBLY_COMP_ID')
    SECT_TYPE_CODE = i.get('SECT_TYPE_CODE')
    CATALOG_KEY_DESC = i.get('CATALOG_KEY_DESC')
    COMP_NAME = i.get('COMP_NAME')
    DESCRIPTION = i.get('DESCRIPTION')
    LENGTH = i.get('LENGTH')
    ID_BODY = i.get('ID_BODY')
    MD_BASE = i.get('MD_BASE')
    MD_TOP = i.get('MD_TOP')
    OD_BODY = i.get('OD_BODY')
    SEQUENCE_NO = i.get('SEQUENCE_NO')
    COMP_TYPE_CODE = i.get('COMP_TYPE_CODE')
    CREATE_USER_ID = i.get('CREATE_USER_ID')
    CREATE_DATE = i.get('CREATE_DATE')
    GRADE = i.get('GRADE')
    JOINTS = i.get('JOINTS')
    CONNECTION_NAME = i.get('CONNECTION_NAME')
    NOMINAL_SIZE = i.get('NOMINAL_SIZE')
    
    asmbly_rows.append({'WELL_ID':WELL_ID,'ASSEMBLY_ID':ASSEMBLY_ID,'ASSEMBLY_COMP_ID':ASSEMBLY_COMP_ID,'SECT_TYPE_CODE':SECT_TYPE_CODE,'CATALOG_KEY_DESC':CATALOG_KEY_DESC,'COMP_NAME':COMP_NAME,
              'DESCRIPTION':DESCRIPTION,'LENGTH':LENGTH,'OD_BODY':OD_BODY,'SEQUENCE_NO':SEQUENCE_NO,'COMP_TYPE_CODE':COMP_TYPE_CODE,'CREATE_USER_ID':CREATE_USER_ID,
               'CREATE_DATE':CREATE_DATE.strip('{').strip('}').strip('ts').strip("'").strip(" '"),'GRADE':GRADE,'JOINTS':JOINTS,'CONNECTION_NAME':CONNECTION_NAME,'NOMINAL_SIZE':NOMINAL_SIZE})
asmbly = pd.DataFrame(asmbly_rows, columns = asmbly_cols).drop_duplicates()
asmbly['LENGTH'] = asmbly['LENGTH'].astype(float).round(2)
asmbly['JOINTS'] = asmbly['JOINTS'].astype(float)
asmbly = pd.merge(asmbly,wellname[['WELL_ID','WELL_LEGAL_NAME']], on = 'WELL_ID')
asmbly = pd.merge(asmbly,asm[['ASSEMBLY_ID','ASSEMBLY_NAME','DESCRIPTION','ENTITY_TYPE']], on = 'ASSEMBLY_ID')
asmbly = asmbly[asmbly['SECT_TYPE_CODE'].isin(sect1)]
asmbly = asmbly.loc[~(asmbly['ASSEMBLY_NAME'].str.contains(asm_name_desc, case=False))]
asmbly = asmbly.loc[~(asmbly['CATALOG_KEY_DESC'].str.contains(ctlg_desc,case=False))]
burried = asmbly.groupby(['WELL_LEGAL_NAME','OD_BODY','CATALOG_KEY_DESC']).agg({'LENGTH':'sum', 'JOINTS':'sum'})
# burried

#Create an export table 
column_names = ['Well Name','WBS-Element','OD','Material description (Dia, wt#, Grade, Conn)',
                'Delivered from Supplier(jts)', 'Delivered from Supplier(ft)', 'Transfer In(jts)', 
                'Transfer In(ft)','Off Load Report(jts)','Off Load Report(ft)','Burried(jts)','Burried(ft)',
                'Burried Thread Loss Footage(Ft)','Transfer Out(jts)','Transfer Out(ft)','Difference(jts)', 'Diference(ft)', 'Comments']
df = pd.DataFrame(columns = column_names)
df_burried = burried.reset_index()
df_burried.rename(columns = {df_burried.columns[2]:'DESCRIPTION'}, inplace = True)
df1 = df_burried.merge(offload.reset_index(),on=['DESCRIPTION'], how='left')
df1['KEYWORD'] = df1.DESCRIPTION + ' ' + df1.OD_BODY
df1['KEYWORD'] = df1['KEYWORD'].str.replace("JOINTS","")
transfer_in = transferdata[transferdata['in/out'] == 'transfer in']
transfer_out = transferdata[transferdata['in/out'] == 'transfer out']
dftransfer = pd.merge(transfer_in, transfer_out, on = ['DESCRIPTION'], how = 'left')
# dftransfer
df1.pop('DESCRIPTION')
df2 = fpd.fuzzy_merge(df1, dftransfer, left_on = ['KEYWORD'], join = 'left-outer',right_on=['DESCRIPTION'],method = 'levenshtein', ignore_case = True,ignore_order_words =True,threshold =0.60)

# df2 = fpd.fuzzy_merge(df1, transfer_out, left_on = ['KEYWORD'], join = 'left-outer',right_on=['DESCRIPTION'],method = 'levenshtein', ignore_case = True,ignore_order_words =True,threshold =0.6)
# df3 = fpd.fuzzy_merge(df2, transfer_in, left_on = ['KEYWORD'], join = 'left-outer',right_on=['DESCRIPTION'],method = 'levenshtein', ignore_case = True,ignore_order_words =True,threshold =0.6)
        
for i,r in df2.iterrows():
    df = df.append({'Well Name': well
                    , 'OD': df2['OD_BODY'].loc[i]
                    ,'WBS-Element': wbs
                    , 'Burried(jts)': df2['JOINTS'].loc[i]
                    ,'Burried(ft)': df2['LENGTH_x'].loc[i]
                    ,'Material description (Dia, wt#, Grade, Conn)': df2['KEYWORD'].loc[i]
                    ,'Off Load Report(jts)': df2['NO_JOINTS'].loc[i]
                    , 'Off Load Report(ft)': df2['LENGTH_y'].loc[i]
                    ,'Transfer In(jts)': df2['QUANTITY_x'].loc[i]
                    ,'Transfer In(ft)': df2['MANUFACTURER_SERIAL_NUMBER_x'].loc[i]
                    ,'Transfer Out(jts)': df2['QUANTITY_y'].loc[i]
                    ,'Transfer Out(ft)': df2['MANUFACTURER_SERIAL_NUMBER_y'].loc[i]
#                     ,'Difference(jts)': '=' + 'D' + str(i) + '+' + 'F' + str(i) + '-' + 'J' + str(i) + '-' + 'M' + str(i)
                    ,'Burried Thread Loss Footage(Ft)': df2['LENGTH_x'].loc[i]*0.01,'Comments': df2['REASON_y'].loc[i]}
                   , ignore_index = True)

df = df.fillna('').sort_values(by=['OD'],ascending = False ).reset_index(drop=True)
df.pop('OD')

#enter the excel formula after the table is sorted
for i,r in df.iterrows():
    df['Difference(jts)'].iloc[i] = '=' + 'D' + str(i+2) + '+' + 'F' + str(i+2) + '-' + 'J' + str(i+2) + '-' + 'M' + str(i+2)
for i,r in df.iterrows():
    df['Diference(ft)'].iloc[i] = '=(' + 'E' + str(i+2) + '+' + 'G' + str(i+2) + ')-(' + 'K' + str(i+2) + '+' + 'L' + str(i+2) +')-' + 'N' + str(i+2)
#  =(E5+G5)-(K5+L5)-N5   

#Name of agents performing transfer
agent_transfer = transfer[["WELL_LEGAL_NAME",'in/out','AGENT',"DESCRIPTION","ORIGIN",'DESTINATION','REASON','QUANTITY','MANUFACTURER_SERIAL_NUMBER']]
agent_transfer.rename(columns = {agent_transfer.columns[8]: 'LENGTH'}, inplace = True)
#casing report creator
casing_detail = asmbly[["ENTITY_TYPE","CATALOG_KEY_DESC","ASSEMBLY_NAME","CREATE_USER_ID","GRADE","JOINTS","CONNECTION_NAME"]]

#export the data into excel format to be reviewed

writer = pd.ExcelWriter(r"C:\Users\\Documents\Converted Files"+file_name+".xlsx",engine='xlsxwriter')
df.to_excel(writer,sheet_name="Report",index=False)
agent_transfer.to_excel(writer, sheet_name = "detail transfer", index = False)
casing_detail.to_excel(writer,sheet_name = "detail burried joints", index = False)
writer.save()


#GL for Casing & Tubing 760215: Casing & TBG 760220

Enter file name: 


In [2]:

reports

,REPORT_JOURNAL_ID,EVENT_ID,DESCRIPTION,CREATE_DATE,ENTITY_TYPE,EVENT_CODE,REPORT_ALIAS
20,KlDMmXABGJ,xwrct,MT RECEIVED 5.5'' CASING,{ts '2019-11-12 15:11:57'},Material Transfer,DON,GEN_TRAN
25,CIiSnNH6J2,eZqTW,5.5 PKR AND 2.875 PROD TBG,{ts '2020-02-13 10:50:04'},Wellbore Equipment,CON,WB_EQUIP
63,HXaDRtZOhS,xwrct,5.5'' OFF-LOAD TALLY,{ts '2019-11-16 10:43:40'},Pipe Tally,DON,GEN_PIPE
67,9NaW657evo,xwrct,5.5 PROD CASING REPORT,{ts '2019-11-21 16:26:05'},Casing,DON,CASING
75,USo0aj2aYG,xwrct,"TRANSFER 5.5"" CASING TO 63H",{ts '2019-11-21 16:56:57'},Material Transfer,DON,GEN_TRAN
122,QlNgPCVy8L,eZqTW,2.875 PROD TBG DELIVERED,{ts '2020-02-12 12:16:08'},Material Transfer,CON,GEN_TRAN
125,VZZHoR8wM2,xwrct,9.625'' OFF LOAD TALLY,{ts '2019-10-16 21:20:08'},Pipe Tally,DON,GEN_PIPE
128,HXmmrvbshy,xwrct,RECIEVE FROM JC MARTIN 63H,{ts '2019-10-21 07:31:53'},Material Transfer,DON,GEN_TRAN
134,BP9SXmA6uP,xwrct,"MATERIAL TRANSFER 9.625"" 27JTS TO JC MARTIN 65H",{ts '2019-10-19 12:03:05'},Material Transfer,DON,GEN_TRAN
135,B2xd2UkSN6,xwrct,"9.625"" SURFACE CASING REPORT",{ts '2019-10-19 17:22:21'},Casing,DON,CASING
